In [1]:
import os
from twitter_to_sqlite import utils
import pandas as pd
import time

In [2]:
auth = json.load(open("../auth/auth.json"))
owner_id = "143058191"
session = utils.session_for_auth(auth)

In [3]:
tweets_df = pd.read_csv("seen.csv")

In [14]:
c = "to_process"
url = f"https://api.twitter.com/1.1/collections/create.json?name={c}"
session.post(url)

<Response [200]>

In [15]:
url = f"https://api.twitter.com/1.1/collections/list.json?user_id={owner_id}"
response = session.get(url)
collections = response.json()["objects"]["timelines"]
for k in collections.keys():
    print(k, collections[k]["name"])
collections_list = [collections[k]["name"] for k in collections.keys()]

custom-1351959730080645120 to_process
custom-1351846441472372737 to_process
custom-1351555076024893440 custom_newsfeed
custom-1351093543385899011 not_relevant


In [9]:
tweets_df

,tweet_id,err_reason
0,1337875267096875015,NaN
1,1335698027537969155,NaN
2,1345699105385959435,not_found
3,1336874318882467842,NaN
4,1337717825629253633,not_found
...,...,...
1595,1337184446789931008,not_found
1596,1341250063876550656,NaN
1597,1336231355110322177,not_found
1598,1344289183456768002,NaN


In [8]:
tweet_list = tweets_df["tweet_id"]

In [10]:
def proccecing_list(collection_id, tweet_list):
    
    collection_id = collection_id
    procc_list = []
    for counter, tweet_id in enumerate(tweet_list):
        if (counter+1) % 20 == 0:
            print(f"{(counter+1)} / {len(tweet_list)}")
        url = f"https://api.twitter.com/1.1/collections/entries/add.json?tweet_id={tweet_id}&id={collection_id}"
        response = session.post(url)
        
    while response.reason != "OK":
        print(response.reason)
        if response.reason == "Too Many Requests":
            sleep = 15
            time.sleep(sleep)
            print(f"Rate limit error - waiting for {sleep} seconds")
        else:
            if "errors" in response.json():
                print(response.json()["errors"][0]["message"])
            elif "error" in response.json():
                print(response.json()["error"])
            else:
                print(response.json())
            raise Exception(f"Status code: {response.status_code}")

        if response.reason == "OK":
            errors = response.json()["response"]["errors"]
            if len(errors) > 0:
                procc_list.append({"tweet_id": tweet_id, "err_reason" : errors[0]["reason"]})
            else:
                procc_list.append({"tweet_id": tweet_id, "err_reason" : "no_errors"})

    df = pd.DataFrame(procc_list)
    print(df["err_reason"].value_counts())
    return df

In [23]:
tweet_list = tweets_df["tweet_id"].tolist()

In [87]:
def err_handling(response, sleep=15):
    while response.reason != "OK":
        print(response.reason)
        if response.reason == "Too Many Requests":
            print(f"Rate limit error - waiting for {sleep} seconds")
            time.sleep(sleep)
        else:
            if "errors" in response.json():
                print(response.json()["errors"][0]["message"])
            elif "error" in response.json():
                print(response.json()["error"])
            else:
                print(response.json())
            raise Exception(f"Status code: {response.status_code}")

In [88]:
collection_id = "custom-1351959730080645120"
procc_list = []
for counter, tweet_id in enumerate(tweet_list):
    if (counter+1) % 20 == 0:
        print(f"{(counter+1)} / {len(tweet_list)}")
    url = f"https://api.twitter.com/1.1/collections/entries/add.json?tweet_id={tweet_id}&id={collection_id}"
    response = session.post(url)
    err_handling(response)
            
    if response.reason == "OK":
        errors = response.json()["response"]["errors"]
        if len(errors) > 0:
            procc_list.append({"tweet_id": tweet_id, "err_reason" : errors[0]["reason"]})
        else:
            procc_list.append({"tweet_id": tweet_id, "err_reason" : "no_errors"})

df = pd.DataFrame(procc_list)
print(df["err_reason"].value_counts())

Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds
Too Many Requests
Rate limit error - waiting for 15 seconds


KeyboardInterrupt: 

In [91]:
response.text

''

In [73]:
response.json()

{'errors': [{'code': 34, 'message': 'Sorry, that page does not exist.'}]}

In [92]:
response = session.post(url)
print(response.status_code)
print(response.headers)
print(response.url)
print(response.history)
print(response.encoding)
print(response.reason)
print(response.status_code)
print(response.cookies)
print(response.elapsed)
print(response.request)


429
{'cache-control': 'no-cache, no-store, must-revalidate, pre-check=0, post-check=0', 'content-length': '0', 'content-security-policy': "default-src 'self'; connect-src 'self'; font-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com data:; frame-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com; img-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com data:; media-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com; object-src 'none'; script-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com; style-src 'self' https://*.twimg.com https://twitter.com https://ton.twitter.com; report-uri https://twitter.com/i/csp_report?a=ORUW2ZLMNFXGK43FOJ3GSY3FFVQXA2I%3D&ro=false;", 'content-type': 'text/html;charset=utf-8', 'date': 'Wed, 20 Jan 2021 19:09:55 GMT', 'expires': 'Tue, 31 Mar 1981 05:00:00 GMT', 'last-modified': 'Wed, 20 Jan 2021 19:09:55 GMT', 'pragma': 'no-cache', 'server

In [59]:
url = f"https://api.twitter.com/1.1/collections/entries/add.json?tweet_id=1337875267096875015&id=custom-1351446441472372737"
response = session.post(url)

In [62]:
response.json()

{'errors': [{'code': 34, 'message': 'Sorry, that page does not exist.'}]}

In [ ]:
_content', 'status_code', 'headers', 'url', 'history',
        'encoding', 'reason', 'cookies', 'elapsed', 'request'

In [11]:
# while respons is not ok = sleep for 60s


In [58]:
df = proccecing_list("custom-1351846445472372737", tweet_list)

0
Not Found


KeyboardInterrupt: 

In [21]:
df.shape

(770, 2)

In [22]:
df2 = proccecing_list("custom-1351839030959288320", tweet_list.tolist()[760:])

20 / 840
40 / 840
60 / 840
80 / 840
100 / 840
120 / 840
140 / 840
160 / 840
180 / 840
200 / 840
220 / 840
240 / 840
260 / 840
280 / 840
300 / 840
320 / 840
340 / 840
360 / 840
380 / 840
400 / 840
420 / 840
440 / 840
460 / 840
480 / 840
500 / 840
520 / 840
540 / 840
560 / 840
580 / 840
600 / 840
620 / 840
640 / 840
660 / 840
680 / 840
700 / 840
720 / 840
740 / 840
760 / 840
780 / 840
800 / 840
820 / 840
840 / 840
                   616
not_found          202
protected_tweet     14
duplicate            8
Name: err_reason, dtype: int64


In [23]:
df

,tweet_id,err_reason
0,1337875267096875015,
1,1335698027537969155,
2,1345699105385959435,not_found
3,1336874318882467842,
4,1337717825629253633,not_found
...,...,...
765,1336759256041840645,
766,755026645069750272,
767,1336457867235364867,
768,1339354804435341314,


In [24]:
df2

,tweet_id,err_reason
0,1340113559712350211,not_found
1,1337484424909209601,duplicate
2,1345334329698877441,duplicate
3,1339020019951349762,duplicate
4,1337920190340681729,not_found
...,...,...
835,1337184446789931008,not_found
836,1341250063876550656,
837,1336231355110322177,not_found
838,1344289183456768002,


In [26]:
frames = [df, df2]

In [27]:
result = pd.concat(frames)

In [31]:
result.drop_duplicates(subset="tweet_id",  inplace=True)

In [32]:
result

,tweet_id,err_reason
0,1337875267096875015,
1,1335698027537969155,
2,1345699105385959435,not_found
3,1336874318882467842,
4,1337717825629253633,not_found
...,...,...
835,1337184446789931008,not_found
836,1341250063876550656,
837,1336231355110322177,not_found
838,1344289183456768002,


In [34]:
result["err_reason"].value_counts()

                   1187
not_found           389
protected_tweet      24
Name: err_reason, dtype: int64

In [35]:
result.to_csv('seen.csv', index=False)